In [87]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm
from sklearn.model_selection import KFold

In [50]:
df = pd.read_csv("data/clean/dataset.csv", index_col='idx')
df.head()

/var/folders/cl/wqxhc67j255bdr0rmv_bl3580000gn/T/ipykernel_61974/1957658052.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/clean/dataset.csv", index_col='idx')


,Start,End,kWh,Revenue,Station,PortNum,PortType,Adapter,Client,Tariff,MaxPower,Fishka,Latitude,Longitude,Type,ChargingTime
idx,,,,,,,,,,,,,,,,
0,2021-01-01 00:05:34,2021-01-01 00:31:22,6.033,27.15,209,3,Type 2 Plug,False,553804960,4.50,NaN,True,50.43104,30.39037,FAST,25.80
1,2021-01-01 01:40:37,2021-01-01 01:42:19,0.890,6.22,111,1,CHAdeMO,False,550843379,6.99,NaN,True,50.48207,30.48725,FAST,1.70
2,2021-01-01 02:12:35,2021-01-01 03:07:36,3.300,14.85,147,1,Type 2,False,540085134,4.50,NaN,True,49.24778,23.85610,SLOW,55.02
3,2021-01-01 02:32:21,2021-01-01 02:57:08,6.880,48.09,111,1,CHAdeMO,False,828881697,6.99,NaN,True,50.48207,30.48725,FAST,24.78
4,2021-01-01 03:39:03,2021-01-01 04:02:05,12.630,56.84,209,3,Type 2 Plug,False,553804960,4.50,NaN,True,50.43104,30.39037,FAST,23.03


In [51]:
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])
df['Weekday'] = df['Start'].dt.day
df['Year'] = df['Start'].dt.year
df['Month'] = df['Start'].dt.month
df['Hour'] = df['Start'].dt.hour
df

,Start,End,kWh,Revenue,Station,PortNum,PortType,Adapter,Client,Tariff,MaxPower,Fishka,Latitude,Longitude,Type,ChargingTime,Weekday,Year,Month,Hour
idx,,,,,,,,,,,,,,,,,,,,
0,2021-01-01 00:05:34,2021-01-01 00:31:22,6.033,27.15,209,3,Type 2 Plug,False,553804960,4.50,NaN,True,50.431040,30.390370,FAST,25.80,1,2021,1,0
1,2021-01-01 01:40:37,2021-01-01 01:42:19,0.890,6.22,111,1,CHAdeMO,False,550843379,6.99,NaN,True,50.482070,30.487250,FAST,1.70,1,2021,1,1
2,2021-01-01 02:12:35,2021-01-01 03:07:36,3.300,14.85,147,1,Type 2,False,540085134,4.50,NaN,True,49.247780,23.856100,SLOW,55.02,1,2021,1,2
3,2021-01-01 02:32:21,2021-01-01 02:57:08,6.880,48.09,111,1,CHAdeMO,False,828881697,6.99,NaN,True,50.482070,30.487250,FAST,24.78,1,2021,1,2
4,2021-01-01 03:39:03,2021-01-01 04:02:05,12.630,56.84,209,3,Type 2 Plug,False,553804960,4.50,NaN,True,50.431040,30.390370,FAST,23.03,1,2021,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371666,2024-10-31 09:34:21,2024-10-31 10:37:24,73.572,1249.99,561,2,CCS 2/GBT DC,False,857230274,16.99,89.0,NaN,50.672440,26.271480,ULTRAFAST,63.05,31,2024,10,9
371667,2024-10-31 09:36:14,2024-10-31 10:01:05,36.793,625.11,550,1,CCS 2,False,503412773,16.99,119.0,NaN,50.319615,31.033856,ULTRAFAST,24.85,31,2024,10,9
371668,2024-10-31 09:36:48,2024-10-31 09:43:43,1.500,23.98,583,2,CHAdeMO,False,548310042,15.99,23.0,True,46.479330,30.690400,FAST,6.92,31,2024,10,9


In [53]:
df = pd.get_dummies(df, columns=['Type'], prefix='Type', drop_first=True)
dummy_cols = [c for c in df.columns if c.startswith(f"Type_")]
df[dummy_cols] = df[dummy_cols].astype(int)

In [59]:
df = pd.get_dummies(df, columns=['PortType'], prefix='PortType', drop_first=True)
dummy_cols = [c for c in df.columns if c.startswith(f"PortType_")]
df[dummy_cols] = df[dummy_cols].astype(int)

In [60]:
df

,Start,End,kWh,Revenue,Station,PortNum,Adapter,Client,Tariff,MaxPower,...,Type_ULTRAFAST,PortType_CCS 2/CCS 1,PortType_CCS 2/GBT DC,PortType_CHAdeMO,PortType_GB/T AC,PortType_GB/T DC,PortType_Type 1,PortType_Type 2,PortType_Type 2 Plug,PortType_Type 2/Type 1
idx,,,,,,,,,,,,,,,,,,,,,
0,2021-01-01 00:05:34,2021-01-01 00:31:22,6.033,27.15,209,3,False,553804960,4.50,NaN,...,0,0,0,0,0,0,0,0,1,0
1,2021-01-01 01:40:37,2021-01-01 01:42:19,0.890,6.22,111,1,False,550843379,6.99,NaN,...,0,0,0,1,0,0,0,0,0,0
2,2021-01-01 02:12:35,2021-01-01 03:07:36,3.300,14.85,147,1,False,540085134,4.50,NaN,...,0,0,0,0,0,0,0,1,0,0
3,2021-01-01 02:32:21,2021-01-01 02:57:08,6.880,48.09,111,1,False,828881697,6.99,NaN,...,0,0,0,1,0,0,0,0,0,0
4,2021-01-01 03:39:03,2021-01-01 04:02:05,12.630,56.84,209,3,False,553804960,4.50,NaN,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371666,2024-10-31 09:34:21,2024-10-31 10:37:24,73.572,1249.99,561,2,False,857230274,16.99,89.0,...,1,0,1,0,0,0,0,0,0,0
371667,2024-10-31 09:36:14,2024-10-31 10:01:05,36.793,625.11,550,1,False,503412773,16.99,119.0,...,1,0,0,0,0,0,0,0,0,0
371668,2024-10-31 09:36:48,2024-10-31 09:43:43,1.500,23.98,583,2,False,548310042,15.99,23.0,...,0,0,0,1,0,0,0,0,0,0


In [63]:
features = ['Hour', 'Month', 'Year', 'Weekday', 'Latitude', 'Longitude', 'Tariff', 'PortNum', 'Type_SLOW', 'Type_ULTRAFAST', 'PortType_CCS 2/CCS 1', 'PortType_CCS 2/GBT DC', 'PortType_CHAdeMO', 'PortType_GB/T AC', 'PortType_GB/T DC', 'PortType_Type 1', 'PortType_Type 2', 'PortType_Type 2 Plug', 'PortType_Type 2/Type 1']
X = df[features]
y = df['kWh']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [65]:
train_preds = lr_model.predict(X_train)
np.sqrt(mse(y_train, train_preds))

14.611976601587356

In [66]:
test_preds = lr_model.predict(X_test)
np.sqrt(mse(y_test, test_preds))

14.618405484776401

In [76]:
dt_model = DecisionTreeRegressor(max_depth=10)
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10)

In [77]:
train_preds = dt_model.predict(X_train)
np.sqrt(mse(y_train, train_preds))

13.886207327880102

In [78]:
test_preds = dt_model.predict(X_test)
np.sqrt(mse(y_test, test_preds))

14.107528601424606

In [88]:
param_grid = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

# Initialize the Decision Tree Regressor
dt_model = DecisionTreeRegressor(random_state=42)

# Set up cross-validation with KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Use GridSearchCV to optimize parameters based on RMSE
grid_search = GridSearchCV(
    estimator=dt_model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',  # Minimize RMSE
    cv=kf,
    n_jobs=-1
)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Best parameters and corresponding RMSE
best_params = grid_search.best_params_
best_rmse = -grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Cross-Validated RMSE:", best_rmse)

# Train the model with the best parameters on the full training set
best_dt_model = grid_search.best_estimator_
best_dt_model.fit(X_train, y_train)

# Evaluate on the test set
test_preds = best_dt_model.predict(X_test)
test_rmse = np.sqrt(mse(y_test, test_preds))
print("Test RMSE:", test_rmse)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}
Best Cross-Validated RMSE: 14.122112532893208
Test RMSE: 14.095436294824696
